In [1]:
import os

# Go outside the src directory
os.chdir("..")
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /Users/sarah_shoilee/codeProjects/stamboekn_KE


In [2]:
image_path = "data/images/NL-HaNA_2.10.50_45_0355.jpg"
image_name = os.path.basename(image_path)
base_name = os.path.splitext(image_name)[0]
base_name

'NL-HaNA_2.10.50_45_0355'

RUN LORE and HTR First:
```bash
cd Image2TableBoundingBoxDetection
cd src 
conda deactivate
conda activate LORE
bash scripts/infer/demo_wireless.sh
```

In [3]:
cells_bounding_box = f"data/tables/cells/center/{image_name}.txt"
cells_structure = f"data/tables/cells/logi/{image_name}.txt"
page_file = f"data/htr/page/{base_name}.xml"
json_file = f"data/tables/json/{image_name}.jsonl"

print(f"\nProcessing image: {image_name}")


Processing image: NL-HaNA_2.10.50_45_0355.jpg


In [4]:
from src.reconstruct_table import main as reconstruct_table
reconstruct_table(cells_bounding_box, cells_structure, page_file, json_file, image_name, wired=True)

Table structure built successfully.
Table reconstructed and saved to data/tables/html/NL-HaNA_2.10.50_45_0355.jpg.html


In [5]:
from Image2Table_LLM.parse import format_td
from Image2Table_LLM.metric import TEDS

def calculate_TEDS(ground_truth_html, predicted_html):
    predicted_html = format_td(predicted_html)
    ground_truth_html = format_td(ground_truth_html)

    teds = TEDS(structure_only=False)
    teds_score = teds.evaluate(ground_truth_html, predicted_html)

    teds_struct = TEDS(structure_only=True)
    teds_struct_score = teds_struct.evaluate(ground_truth_html, predicted_html)
    
    print(f"TEDS: {teds_score:.4f}")
    print(f"TEDS-Struct: {teds_struct_score:.4f}")

    return teds_score, teds_struct_score

In [6]:
with open(os.path.join("data/tables/html/", image_name+'.html'), 'r', encoding='utf-8') as f:
    constructed_html = f.read()

with open(os.path.join("data/labels", image_name.replace('.jpg', '.html')), 'r', encoding='utf-8') as f:
    label_html = f.read()

calculate_TEDS(label_html, constructed_html)

TEDS: 0.4946
TEDS-Struct: 0.6667


(0.49464379570691897, 0.6666666666666667)

### Information Extraction

In [7]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(constructed_html, 'html.parser')

rows = soup.find_all('tr')

# --- Step 1: make a grid keeping track of rowspans ---
logical_rows = []
rowspans = {}  # {(col_idx): [remaining_rows, cell_content]}

for r_idx, tr in enumerate(rows):
    # Start with any carried over rowspans from previous rows
    current_row = []
    to_remove = []

    # Fill carried-down cells first
    for col_idx, (remaining, cell) in rowspans.items():
        current_row.append(cell)
        rowspans[col_idx][0] -= 1
        if rowspans[col_idx][0] <= 0:
            to_remove.append(col_idx)
    for col_idx in to_remove:
        del rowspans[col_idx]

    # Now add new cells from this row
    c_idx = 0
    for td in tr.find_all('td'):
        while any(k == c_idx for k in rowspans):  # skip columns occupied by carried cells
            c_idx += 1
        text = td.get_text(" ", strip=True)
        cell_id = td.get('id')
        r_idx = int(td.get('row', r_idx))
        c_idx = int(td.get('col', c_idx))
        rowspan = int(td.get('rowspan', 1))
        colspan = int(td.get('colspan', 1))

        cell_data = {
            'text': text,
            'id': cell_id,
            'row': r_idx,
            'col': c_idx,
            'rowspan': rowspan,
            'colspan': colspan
        }

        # Place cell in current row
        current_row.append(cell_data)

        # Store for future rows if rowspan > 1
        if rowspan > 1:
            rowspans[c_idx] = [rowspan - 1, cell_data]

        c_idx += colspan

    logical_rows.append(current_row)


with open("output.txt", "w+", encoding="utf-8") as f:
    for i, row in enumerate(logical_rows):
        f.write(f"Row {i+1}:\n")
        for cell in row:
            f.write(
                    f"  id={cell.get('id')}, "
                    f"row={cell.get('row')}, "
                    f"col={cell.get('col')}, "
                    f"text={cell.get('text')}\n"
                )
                


In [8]:
import re
import json
from src.person_info_extraction import extract_info_regex, extract_info_LLM

persons = []

for i, row in enumerate(logical_rows):
    person = {}
        
    # print(generate_prompt(cells))
    person = json.loads(extract_info_LLM(row))
    if all(v['value']==None for v in person.values()):
        continue

    if person:
        persons.append(person)

json_obj = {"persons": persons}
print(json.dumps(json_obj, indent=2, ensure_ascii=False))

{
  "persons": [
    {
      "vader": {
        "value": "Jacobus",
        "cell": "4"
      },
      "moeder": {
        "value": "Catharina Gullart",
        "cell": "4"
      },
      "geboorte_datum": {
        "value": "9 october 1775",
        "cell": "4"
      },
      "geboorte_plaats": {
        "value": "Schoonhoven",
        "cell": "4"
      },
      "laatste_woonplaats": {
        "value": "Buille",
        "cell": "4"
      }
    },
    {
      "vader": {
        "value": "Jacobus",
        "cell": "4"
      },
      "moeder": {
        "value": "Catharina Gullart",
        "cell": "4"
      },
      "geboorte_datum": {
        "value": "9 october 1775",
        "cell": "4"
      },
      "geboorte_plaats": {
        "value": "Schoonhoven",
        "cell": "4"
      },
      "laatste_woonplaats": {
        "value": "Buille",
        "cell": "4"
      }
    },
    {
      "vader": {
        "value": null,
        "cell": null
      },
      "moeder": {
        "value": "M

In [9]:
with open(f"data/json/{image_name}.json", "w", encoding='utf-8') as json_file:
    json.dump(json_obj, json_file, ensure_ascii=False, indent=2)#!/usr/bin/env python3


In [10]:
from src.metrics import best_match_similarity
# from src.metrics import calculate_normalized_information_distance

with open(f"data/json/{image_name}.json", 'r', encoding='utf-8') as f:
    constructed_html = json.load(f)
with open(os.path.join("data/labels", image_name.replace('.jpg', '.json')), 'r', encoding='utf-8') as f:
    label_html = json.load(f)

# calculate_normalized_information_distance(constructed_html, label_html)
best_match_similarity(constructed_html.get("persons", []), label_html.get("persons", []))


np.float64(0.5414404223227752)

### KG construction

In [11]:
# load json from file
with open(f"data/json/{image_name}.json", "r", encoding="utf-8") as f:
    json_obj = json.load(f)

In [12]:
# CONSTRUCT ASSERSION TRIPLES
from rdflib import Graph, ConjunctiveGraph, Namespace, URIRef, Literal, RDF

FOAF = Namespace("http://xmlns.com/foaf/0.1/")
EX = Namespace("http://example.org/ontology/")
PROV = Namespace("http://www.w3.org/ns/prov#")

cg = ConjunctiveGraph()
cg.bind("foaf", FOAF)
cg.bind("ex", EX)
cg.bind("prov", PROV)

# Mapping from json keys to RDF predicates
predicate_map = {
    "vader": EX.vader,
    "moeder": EX.moeder,
    "geboorte_datum": EX.geboorteDatum,
    "geboorte_plaats": EX.geboortePlaats,
    "laatste_woonplaats": EX.laatsteWoonplaats
}

for idx, person in enumerate(json_obj["persons"], start=1):
    person_uri = URIRef(f"http://example.org/person/{idx}")
    assertion_graph_uri = URIRef("http://example.org/assertion")
    assertion_graph = Graph(store=cg.store, identifier=assertion_graph_uri)
    assertion_graph.add((person_uri, RDF.type, FOAF.Person))

    provenance_graph_uri = URIRef("http://example.org/provenance")
    provenance_graph = Graph(store=cg.store, identifier=provenance_graph_uri)

    for key, value_dict in person.items():
        value = value_dict["value"]
        cell_id = value_dict["cell"]
        
        predicate = predicate_map.get(key)
        
        # if cell_id is null, no named graph can be created
        if not cell_id:
            assertion_graph.add((person_uri, predicate, Literal(value)))
            continue
        
        if predicate:
            # Named graph for each cell
            graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            ng = Graph(store=cg.store, identifier=graph_uri)
            ng.add((person_uri, predicate, Literal(value)))
            

cg.serialize(f"data/triples/{image_name.replace('.jpg','')}_assersion.trig", format='trig')


/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_74602/2391772286.py:8: DeprecationWarning: ConjunctiveGraph is deprecated, use Dataset instead.
  cg = ConjunctiveGraph()


<Graph identifier=N52e315c8a22e473bb5e138e688a31ef9 (<class 'rdflib.graph.ConjunctiveGraph'>)>

In [13]:
end_time = "2025-09-01T12:00:00Z"  # Example end time, replace with actual time if needed
start_time = "2025-09-01T10:00:00Z"  # Example start time, replace with actual time if needed

def add_provenance_graph(html_path, coordinate_path, stamboek_nummer=image_name):
    import pandas as pd

    # csv read file
    with open(coordinate_path, "r") as coordinate_lines:
        coordinate_lines = coordinate_lines.readlines()
    # df = pd.read_csv(coordinate_path, header=None)
    
    with open(html_path, 'r', encoding='utf-8') as f:
        llm_html = f.read()

    soup = BeautifulSoup(llm_html, "html.parser")

    EX = Namespace("http://example.org/ontology/")
    IMG = Namespace("http://example.org/image_ontology/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")  
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    PROV = Namespace("http://www.w3.org/ns/prov#")
    CSVW = Namespace("http://www.w3.org/ns/csvw#")

    # Create RDF graph
    g = Graph()
    g.bind("ex", EX)
    g.bind("img", IMG)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("prov", PROV)
    g.csvw = ("csvw", CSVW)

    for cell in soup.find_all("td"):
        cell_id = cell.get('id')
        rows = cell.get('row')
        cols = cell.get('col')

        if not cell_id:
            # when cell is None or Merged
            continue
        
        try:
            coords_line = coordinate_lines[int(cell_id)]
            coords_points = coords_line.strip().split(";")
        except Exception as e:
            print(f"Error retrieving coordinates for cell {cell_id}: {e}")
            continue
        
        # cell uri
        named_graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
        cell_uri = URIRef(f"http://example.org/id/{cell_id}")
        provenance_graph.add((named_graph_uri, PROV.wasDerivedFrom, cell_uri))
        g.add((cell_uri, RDF.type, PROV.Entity))
        g.add((cell_uri, RDFS.label, Literal(f"Cell {cell_id} from {stamboek_nummer}")))
            
        g.add((cell_uri, RDF.type, CSVW.Cell))
        g.add((cell_uri, CSVW.rowNumber, Literal(rows)))
        g.add((cell_uri, CSVW.columnNumber, Literal(cols)))
        g.add((cell_uri, EX.ImageRegion, Literal(coords_points)))

        # agents
        agent_1 = URIRef("http://example.org/agent/1")
        g.add((agent_1, RDF.type, PROV.Agent))
        g.add((agent_1, RDFS.label, Literal("Sarah Shoilee")))
        g.add((named_graph_uri, PROV.wasAttributedTo, agent_1))
        project_agent = URIRef("http://example.org/agent/2")
        g.add((project_agent, RDF.type, PROV.Agent))
        g.add((project_agent, RDFS.label, Literal("Pressing Matter Project")))
        g.add((agent_1, PROV.actedOnBehalfOf, project_agent))

        # activity
        stamboekenKGConstructionactivity = URIRef(f"http://example.org/activity/stamboekenKGConstructionactivity/{cell_id}")
        tableConstructionactivity = URIRef(f"http://example.org/activity/TableExtraction/{cell_id}")
        informationExtractionactivity = URIRef(f"http://example.org/activity/InformationExtraction/{cell_id}")
        KGConstructionactivity = URIRef(f"http://example.org/activity/KGConstruction/{cell_id}")
            
        g.add((stamboekenKGConstructionactivity, RDF.type, PROV.Activity))
        g.add((named_graph_uri, PROV.wasGeneratedBy, stamboekenKGConstructionactivity))
        g.add((stamboekenKGConstructionactivity, PROV.wasAssociatedWith, agent_1))
        g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, tableConstructionactivity))
        g.add((tableConstructionactivity, RDF.type, PROV.Activity))
        g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, informationExtractionactivity))
        g.add((informationExtractionactivity, RDF.type, PROV.Activity))
        g.add((informationExtractionactivity, PROV.used, cell_uri))
        g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, KGConstructionactivity))
        g.add((KGConstructionactivity, RDF.type, PROV.Activity))
        g.add((KGConstructionactivity, PROV.used, cell_uri))

        g.add((stamboekenKGConstructionactivity,PROV.endedAtTime, Literal(end_time)))
        g.add((stamboekenKGConstructionactivity,PROV.startedAtTime, Literal(start_time)))

        # Create a Table instance URI
        table_uri = URIRef(f"http://example.org/Table/{cell_id}")
        g.add((table_uri, RDF.type, PROV.Entity))
        g.add((table_uri, RDF.type, CSVW.Table))
        g.add((table_uri, PROV.wasGeneratedBy, tableConstructionactivity))
        g.add((cell_uri, PROV.wasDerivedFrom, table_uri))
            
        # stamboeken
        stamboek_uri = URIRef(f"http://example.org/stamboek/{stamboek_nummer}")
        g.add((stamboek_uri, RDF.type, PROV.Entity))
        g.add((tableConstructionactivity, PROV.used, stamboek_uri))
        g.add((table_uri, PROV.wasDerivedFrom, stamboek_uri))
        national_archives = URIRef("http://example.org/agent/3")
        g.add((national_archives, RDF.type, PROV.Agent))
        g.add((national_archives, RDFS.label, Literal("Nationaal Archief")))
        g.add((stamboek_uri, PROV.wasAttributedTo, national_archives))
    
    g.serialize(f"data/triples/{image_name.replace('.jpg','')}_provenance.ttl", format='ttl')


In [14]:
csv_path = os.path.join("data/tables/cells/center", image_name+'.txt')
table_path = os.path.join("data/tables/html", image_name+'.html')

add_provenance_graph(table_path, csv_path, stamboek_nummer=image_name)